### Functor

La clase `Functor` está definida como:

In [1]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

Es decir, define cómo aplicar un "mapeo" de `f a` a `f b`

Para el árbol:

In [34]:
data Tree a = Nil | Node (Tree a) a (Tree a) deriving Show

instance Functor Tree where
    fmap f Nil = Nil
    fmap f (Node left x right) = Node (fmap f left) (f x) (fmap f right)

In [35]:
t = Node (Node Nil 2 Nil) 1 (Node (Node Nil 4 Nil) 3 (Node Nil 5 Nil))
fmap (\x -> x*x) t

Node (Node Nil 4 Nil) 1 (Node (Node Nil 16 Nil) 9 (Node Nil 25 Nil))

## Monad

In [ ]:
class Applicative m => Monad where
    return :: a -> m a 
    (>>=) :: m a -> (a -> m b) -> m b -- bind == join . fmap

# Writer

In [2]:
f :: Double -> Double
f x = x^2

g :: Double -> Double
g x = 2 * log x

In [59]:
f 5
g 10

25.0

4.605170185988092

Supongamos que queremos loguear cada vez que se llama a las funciones:

In [60]:
fLog :: Double -> (Double, String)
fLog x = (f x, "Llame a f")

gLog :: Double -> (Double, String)
gLog x = (g x, "Llame a g")

In [61]:
fLog 5
gLog 25

(25.0,"Llame a f")

(6.437751649736401,"Llame a g")

¿Qué pasa si queremos componer `f` con `g` __y__ loggear al mismo tiempo?

¡Los tipos de `fLog` y `gLog` no son compatibles!

In [62]:
fgLog = fLog . gLog

<interactive>:1:16: error:
    • Couldn't match type ‘(Double, String)’ with ‘Double’
      Expected type: Double -> Double
        Actual type: Double -> (Double, String)
    • In the second argument of ‘(.)’, namely ‘gLog’
      In the expression: fLog . gLog
      In an equation for ‘fgLog’: fgLog = fLog . gLog

In [63]:
fgLog = fLog . fst . gLog

In [64]:
fgLog 5

(10.361161575920939,"Llame a f")

In [3]:
fLog :: (Double, [String]) -> (Double, [String])
fLog (x,s) = (f x, s ++ ["Llame a f"])

gLog :: (Double, [String]) -> (Double, [String])
gLog (x,s) = (g x, s ++ ["Llame a g"])

In [5]:
fLog . gLog $ (2,[])

(1.9218120556728056,["Llame a g","Llame a f"])

In [87]:
type Log = [String]

fLog :: Double -> (Double, Log)
fLog x = (f x, ["Llame a f"])

gLog :: Double -> (Double, Log)
gLog x = (g x, ["Llame a g"])

#### ¿Cómo podemos combinar las dos operaciones y concatenar los logs?

Primero creamos una función `fmapW` que nos permite aplicar otra función a un resultado de tipo `(Double, Log)`, sólo al `Double`, y dejando el `Log` intacto:

In [88]:
fmapW :: (Double -> a) -> (Double, Log) -> (a, Log)
fmapW f (x,s) = (f x,s)

In [89]:
fmapW f (2,["log"])

(4.0,["log"])

¿Y si mapeamos una función que devuelva otro `(Double, Log)`?

In [90]:
r = gLog 2

In [91]:
fmapW fLog r

((1.9218120556728056,["Llame a f"]),["Llame a g"])

Nosotros buscamos que quede `(1.92...,["Llame a g", "Llame a f"])`. Para esto, creamos la función `joinW`:

In [92]:
joinW :: ((Double, Log), Log) -> (Double, Log)
joinW ((x,s),ss) = (x, ss ++ s)

In [95]:
fmapW fLog r

((1.9218120556728056,["Llame a f"]),["Llame a g"])

In [97]:
joinW . fmapW fLog $ r

(1.9218120556728056,["Llame a g","Llame a f"])

Esta función se llama `bindW`:

In [99]:
bindW :: (Double, Log) -> (Double -> (Double, Log)) -> (Double, Log)
bindW (x,s) f = joinW . fmapW f $ (x,s)

In [103]:
r = (2,[]) `bindW` gLog
r

(1.3862943611198906,["Llame a g"])

In [104]:
r `bindW` fLog

(1.9218120556728056,["Llame a g","Llame a f"])

Para no tener que poner el valor inicial del Log "a mano", creamos la función `returnW`:

In [107]:
returnW :: Double -> (Double, Log)
returnW x = (x, [])

In [110]:
returnW 2 `bindW` gLog `bindW` fLog

(1.9218120556728056,["Llame a g","Llame a f"])

Acabamos de construir el `Monad` __`Writer`__

In [124]:
import Control.Monad.Writer

fLog :: Double -> Writer Log Double
fLog x = writer(f x,["Llame a f"])

gLog :: Double -> Writer Log Double
gLog x = writer(g x, ["Llame a g"])

In [125]:
runWriter(return 2 >>= gLog >>= fLog)

Line 1: Monad law, left identity
Found:
return 2 >>= gLog
Why not:
gLog 2

(1.9218120556728056,["Llame a g","Llame a f"])

In [126]:
runWriter(gLog 2 >>= fLog)

(1.9218120556728056,["Llame a g","Llame a f"])

### Notación __`do`__

In [127]:
gYf :: Writer Log Double
gYf = do
    x <- gLog 2
    fLog x

In [129]:
runWriter gYf

(1.9218120556728056,["Llame a g","Llame a f"])

En general, los `Monad` sirven para:
* Combinar cálculos, cómputos, de alguna forma determinada: _"programmable semicolons"_
    * Writer
    * State: Mantener implícitamente un estado
    * Reader: Pasar un mismo parámetro a varias funciones
    * List
    * Error
    * Maybe
    * IO
    * ...
* Escribir código más parecido a los lenguajes imperativos, con la notación __`do`__


# State

In [1]:
push' x xs = ((),x:xs)

In [2]:
pop' (x:xs) = (x,xs)

In [3]:
(r,s) = push' 5 []
(r',s') = push' 6 s
(r'',s'') = push' 7 s'
(r''',s''') = pop' s''

In [4]:
r'''
s'''

7

[6,5]

In [8]:
:t foldl

foldl :: forall (t :: * -> *) b a. Foldable t => (b -> a -> b) -> b -> t a -> b

In [29]:
push'' x s = snd (push' x s)

In [30]:
push'' 2 []

[2]

In [32]:
foldl (flip push'') [] [5,6,7]

[7,6,5]

In [127]:
import Control.Monad.State

-- newtype State s a = State { runState :: s -> (a,s) }  

--state :: (s -> (a, s)) -> State s a

pop :: Stat
   e [a] a
pop = state (\(x:xs) -> (x,xs))

push :: a -> State [a] ()
push x = state (\xs -> ((),x:xs))

In [116]:
ops = evalState (push 5 >> push 6 >> push 7 >>  pop)
ops []

7

In [11]:
ops :: State [Int] Int
ops = do 
        push 5
        push 6
        push 7
        pop
op = runState (ops)

In [12]:
op []

(7,[6,5])